In [1]:
#install the Pyterrier framework
!pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00

In [2]:
import pyterrier as pt
if not pt.started():
  pt.init()

<ipython-input-2-c43b832924c1>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
<ipython-input-2-c43b832924c1>:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [3]:
#install the Arabic stop words library
!pip install Arabic-Stopwords

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.0 MB/s eta 0:00:00


In [4]:
#we need to import the following libraries.
import pandas as pd
#to display the full text on the notebook without truncation
pd.set_option('display.max_colwidth', 150)
import numpy as np
import re
from snowballstemmer import stemmer
from tqdm import tqdm
import arabicstopwords.arabicstopwords as stp

## Data Cleaning

In [5]:
#removing Stop Words function
def remove_stopWords(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

#a function to normalize the tweets
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

#define the stemming function
ar_stemmer = stemmer("arabic")
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

## Upload already created indexe

In [6]:
# %rm -rf IR-in-Arabic
# %rm -rf evetarIndex
!git clone https://github.com/telsayed/IR-in-Arabic.git
!unzip IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip -d evetarIndex
!ls evetarIndex

Cloning into 'IR-in-Arabic'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 710 (delta 12), reused 0 (delta 0), pack-reused 661 (from 1)
Receiving objects: 100% (710/710), 43.94 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Updating files: 100% (39/39), done.
Archive:  IR-in-Arabic/Summer2021/data/EveTAR/evetarIndex.zip
  inflating: evetarIndex/data.direct.bf  
  inflating: evetarIndex/data.document.fsarrayfile  
  inflating: evetarIndex/data.inverted.bf  
  inflating: evetarIndex/data.lexicon.fsomapfile  
  inflating: evetarIndex/data.lexicon.fsomaphash  
  inflating: evetarIndex/data.lexicon.fsomapid  
  inflating: evetarIndex/data.meta.idx  
  inflating: evetarIndex/data.meta.zdata  
  inflating: evetarIndex/data.properties  
data.direct.bf		   data.lexicon.fsomapfile  data.meta.idx
data.document.fsarrayfile  data.lexicon.fsomaphash  data.meta.zdata
data.inverte

Next, we will load our index. We jus need the data.properties file to load our index.

In [7]:
#we will load the index
index_ref = pt.IndexRef.of("./evetarIndex/data.properties")
index = pt.IndexFactory.of(index_ref)

### **Vector space model retrieval**
We will use BatchRetrieve Pyterrier class for retrieval and TF-IDF as the weighting model.

In [8]:
#set up our retieval model by specifing TF_IDF as wmodel and limiting the number of retrieved results for each query top 10 documents
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

<ipython-input-8-daf647cebb7a>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)


Test retrieval by simple query

In [9]:
#we need to process the query also as we did for documents
def preprocess(sentence):
  # apply preprocessing steps on the given sentence
  # students ToDo .....
  # here you should write your code
  return sentence

query="العربية"
query = preprocess(query)
#we will call the search function using our retrieval model we set up above
results=tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query


## Reading data from multiple data files

In [10]:
dataset_links=["https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-01.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-02.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-03.txt",
               "https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/tweets/evetar-q-04.txt"]

full_data=pd.DataFrame()
for i in tqdm(range(len(dataset_links))):
    tweets=pd.read_csv(dataset_links[i], sep='\t')
    full_data=pd.concat([full_data,tweets],ignore_index=True)
full_data.reset_index(inplace=True,drop=True)
#the docno will be our tweetID
full_data["docno"]=full_data["tweetID"].astype(str)
#select tweet text for the tweets retrieved only
full_data[full_data["docno"].isin(results["docno"].tolist())]


100%|██████████| 4/4 [00:01<00:00,  2.19it/s]


,tweetID,tweetText,docno


Let's try another query.

In [11]:
#we need to process the term also as we did for documents
query="أمريكا دولار"
#preprocess
query = preprocess(query)
#we will call the search function using our retrieval model we set up above
results=tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,37609,559596479149404160,0,7.476369,أمريكا دولار
1,1,158,550091879430512641,1,6.727604,أمريكا دولار
2,1,1532,550629823450591233,2,6.415182,أمريكا دولار
3,1,17911,553526230390030336,3,6.415182,أمريكا دولار
4,1,1312,550563173074362368,4,6.130490,أمريكا دولار
5,1,10,549817806028804096,5,5.943831,أمريكا دولار
6,1,33433,557557782518693889,6,5.943831,أمريكا دولار
7,1,2900,551206536651866112,7,5.869992,أمريكا دولار
8,1,36255,558735802893484033,8,5.869992,أمريكا دولار
9,1,33115,557525142096859136,9,5.698637,أمريكا دولار


Let's check our results

In [12]:
# retrieve the tweets text for the retrieved tweets just to check our results
full_data[full_data["docno"].isin(results["docno"].tolist())]

,tweetID,tweetText,docno
10,549817806028804096,فرص رائعة للاستثمارفي أوروبا خصوصا للدول التي ترتبط عملتها بالدولار الأمريكي حيث فرق السعرأكثر من 10% عن معدل سعر الصرف اليورو مقابل الدولار,549817806028804096
158,550091879430512641,@EhabEino نو يورو نو يورو دولارت بس 😂😂,550091879430512641
1312,550563173074362368,@Mr_w3d معي 60 دولار كاش يو وجهازي سوني 3,550563173074362368
1532,550629823450591233,السلام عليكم كود سوني بلاش ٢٠ دولار http://t.co/pzT3yQjNkX,550629823450591233
2900,551206536651866112,مليون دولار لصيانة وترميم مسجد الملك فيصل في باكستان,551206536651866112
17911,553526230390030336,@suhaildarraj رسوم خدمة فتح حساب 550 دولار ؟؟؟؟؟,553526230390030336


load queries (topics titles) that are already defined and released with EveTAR dataset

In [13]:
#read the topics file from Github and use the titles as queries
topics=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/topics.txt", sep='\t',names=['data'])
queries=[]
qid=[]
#we will get the queries and their ids from the topics file
for i in range(len(topics)):
    splitted=topics["data"][i:i+1][i].split(" ")
    if splitted[0]=="<title>":
       queries.append(' '.join(splitted[1:]))
    if splitted[0]=="<num>":
       qid.append(splitted[2])

queriesDF=pd.DataFrame()
queriesDF["qid"]=qid
queriesDF["raw_query"]=queries
#remove the stopwords from queries, do normalization, and apply stemming
queriesDF["query"]=queriesDF["raw_query"].apply(preprocess)

queriesDF

,qid,raw_query,query
0,E01,تفجير انتحاري في اب,تفجير انتحاري في اب
1,E02,ليتوانيا تستخدم اليورو بدل الليتاس,ليتوانيا تستخدم اليورو بدل الليتاس
2,E03,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية,فلسطين تطلب الانضمام للمحكمة الجنائية الدولية
3,E04,وفاة أبو أنس الليبي في نيويورك,وفاة أبو أنس الليبي في نيويورك
4,E05,اختراق كوريا الشمالية حسابات سوني,اختراق كوريا الشمالية حسابات سوني
5,E06,بناء أول كنيسة في إسطنبول منذ قرن,بناء أول كنيسة في إسطنبول منذ قرن
6,E07,باباندريو يؤسس حزب جديد,باباندريو يؤسس حزب جديد
7,E08,بوكو حرام تخطف شباب في نيجريا,بوكو حرام تخطف شباب في نيجريا
8,E09,سيطرة بوكو حرام على قاعدة عسكرية في نيجيريا,سيطرة بوكو حرام على قاعدة عسكرية في نيجيريا
9,E10,مقتل مسلحين في غارات في باكستان,مقتل مسلحين في غارات في باكستان


We can retrieve the relevant documents to a set of queries using the **transform** function. We will use the set of queries we prepared earlier. The input should be a dataframe containing the **qid** and **query**. The function will return the same dataframe but with an extra 4 columns **docid**, **docno**, **rank**, and **score**.

In [14]:
#the queries dataframe should have qid and query columns
tfidf_res=tfidf_retr.transform(queriesDF)
tfidf_res

,qid,docid,docno,rank,score,raw_query,query
0,E01,1094,550485444136534016,0,8.817682,تفجير انتحاري في اب,تفجير انتحاري في اب
1,E01,537,550292122801557504,1,8.729337,تفجير انتحاري في اب,تفجير انتحاري في اب
2,E01,569,550299899191775232,2,8.729337,تفجير انتحاري في اب,تفجير انتحاري في اب
3,E01,21506,553981982497964033,3,8.600324,تفجير انتحاري في اب,تفجير انتحاري في اب
4,E01,22468,554057756781858816,4,8.586513,تفجير انتحاري في اب,تفجير انتحاري في اب
...,...,...,...,...,...,...,...
455,E65,45437,560893914786197504,5,8.790802,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
456,E65,45445,560894631294939136,6,8.790802,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
457,E65,45455,560895285035933699,7,8.790802,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
458,E65,45458,560895659381780480,8,8.790802,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي


### **BM25**

We will initialize our BM25 retrieval model by using the BatchRetrieve class and setting the weighting model to BM25.

In [15]:
#specify BM25 as wmodel
bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=10)
#the queries dataframe should have qid and query columns
bm25_res=bm25_retr.transform(queriesDF)
bm25_res

<ipython-input-15-8ea042604052>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25_retr = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"},num_results=10)


,qid,docid,docno,rank,score,raw_query,query
0,E01,1094,550485444136534016,0,15.906980,تفجير انتحاري في اب,تفجير انتحاري في اب
1,E01,22468,554057756781858816,1,15.670246,تفجير انتحاري في اب,تفجير انتحاري في اب
2,E01,537,550292122801557504,2,15.619904,تفجير انتحاري في اب,تفجير انتحاري في اب
3,E01,569,550299899191775232,3,15.619904,تفجير انتحاري في اب,تفجير انتحاري في اب
4,E01,21506,553981982497964033,4,15.491553,تفجير انتحاري في اب,تفجير انتحاري في اب
...,...,...,...,...,...,...,...
455,E65,45437,560893914786197504,5,16.048027,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
456,E65,45445,560894631294939136,6,16.048027,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
457,E65,45455,560895285035933699,7,16.048027,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي
458,E65,45458,560895659381780480,8,16.048027,إعادة تشكيل مجلس الوزراء السعودي,إعادة تشكيل مجلس الوزراء السعودي


### **Evaluating our results**
To evaluate the results we need qrels (relevance judgements). The qrels should be in [TREC format](https://trec.nist.gov/).

In [16]:
qrels=pd.read_csv("https://raw.githubusercontent.com/telsayed/IR-in-Arabic/master/Summer2021/data/EveTAR/qrels.txt", sep='\t',names=['qid','Q0','docno','label'])
qrels['docno']=qrels['docno'].astype(str)
# qrels are in TREC format
qrels = qrels[qrels["docno"].isin(full_data["docno"].tolist())] # to choose qrels for the chosen 50k documents
qrels

,qid,Q0,docno,label
0,E01,Q0,549711593487888387,0
1,E01,Q0,549719610459967488,0
2,E01,Q0,549720880717508608,0
4,E01,Q0,549780004230283264,0
7,E01,Q0,549880138033135617,0
...,...,...,...,...
32961,E39,Q0,553776977346564096,0
32962,E39,Q0,553782643968323584,0
32963,E39,Q0,553782692463276032,0
32964,E39,Q0,553782727858585600,0


To evaluate our results we will use Pyterrier Utils.evaluate function. This function take the results and the qrels dataframe containing three columns which are **qid, docno, label.**

You can add the following parameters:


*   **metrics**: default = ["map", ndcg"], select the evaluation metrics

*   **perquery**: default = False, select whether to show the mean of the metrics or the metrics for each queryList item





In [18]:
# Here, we are evaluating TF_IDF retrieval model
eval = pt.Utils.evaluate(tfidf_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])
eval

<ipython-input-18-0df2f4ebbce2>:2: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(tfidf_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])


{'map': 0.013927668072042102,
 'R@5': 0.00946378524821897,
 'R@10': 0.018447992781532037,
 'R@15': 0.018447992781532037,
 'R@20': 0.018447992781532037,
 'R@30': 0.018447992781532037,
 'R@100': 0.018447992781532037,
 'R@200': 0.018447992781532037,
 'R@500': 0.018447992781532037,
 'R@1000': 0.018447992781532037,
 'P@5': 0.2969696969696969,
 'P@10': 0.3060606060606061,
 'P@15': 0.20404040404040408,
 'P@20': 0.15303030303030304,
 'P@30': 0.10202020202020204,
 'P@100': 0.030606060606060612,
 'P@200': 0.015303030303030306,
 'P@500': 0.006121212121212123,
 'P@1000': 0.0030606060606060614}

In [19]:

# Here, we are evaluating BM25 retrieval model
eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])
eval

<ipython-input-19-521b5aaca114>:2: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']],metrics=["map","recall","P"])


{'map': 0.013984761879582173,
 'R@5': 0.00946378524821897,
 'R@10': 0.01874532501349325,
 'R@15': 0.01874532501349325,
 'R@20': 0.01874532501349325,
 'R@30': 0.01874532501349325,
 'R@100': 0.01874532501349325,
 'R@200': 0.01874532501349325,
 'R@500': 0.01874532501349325,
 'R@1000': 0.01874532501349325,
 'P@5': 0.2969696969696969,
 'P@10': 0.3121212121212122,
 'P@15': 0.20808080808080812,
 'P@20': 0.1560606060606061,
 'P@30': 0.10404040404040406,
 'P@100': 0.03121212121212122,
 'P@200': 0.01560606060606061,
 'P@500': 0.006242424242424244,
 'P@1000': 0.003121212121212122}

In [26]:
from pyterrier.measures import *

# Here, we are evaluating BM25 retrieval model
eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']], metrics =[P@1, P@5,P@10,NDCG@5, R@20,R@50,R@100],perquery=False)
eval

<ipython-input-26-d0e41321d967>:4: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']], metrics =[P@1, P@5,P@10,NDCG@5, R@20,R@50,R@100],perquery=False)


{'P@1': 0.3333333333333333,
 'P@5': 0.2969696969696969,
 'P@10': 0.3121212121212122,
 'nDCG@5': 0.30490565993692625,
 'R@20': 0.01874532501349325,
 'R@50': 0.01874532501349325,
 'R@100': 0.01874532501349325}

In [27]:
# Here, we are evaluating BM25 retrieval model
eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']], metrics =[P@1, P@5,P@10,NDCG@5, R@20,R@50,R@100],perquery=True)
eval

<ipython-input-27-3d1c93c84169>:2: DeprecationWarning: Call to deprecated function (or staticmethod) evaluate. (Use pt.Evaluate instead) -- Deprecated since version 0.9.
  eval = pt.Utils.evaluate(bm25_res,qrels[['qid','docno','label']], metrics =[P@1, P@5,P@10,NDCG@5, R@20,R@50,R@100],perquery=True)


defaultdict(dict,
            {'E01': {'P@1': 1.0,
              'P@5': 0.6,
              'P@10': 0.5,
              'R@20': 0.008605851979345954,
              'R@50': 0.008605851979345954,
              'R@100': 0.008605851979345954,
              'nDCG@5': 0.6548086577531307},
             'E02': {'P@1': 0.0,
              'P@5': 0.0,
              'P@10': 0.0,
              'R@20': 0.0,
              'R@50': 0.0,
              'R@100': 0.0,
              'nDCG@5': 0.0},
             'E03': {'P@1': 0.0,
              'P@5': 0.0,
              'P@10': 0.1,
              'R@20': 0.0029411764705882353,
              'R@50': 0.0029411764705882353,
              'R@100': 0.0029411764705882353,
              'nDCG@5': 0.0},
             'E04': {'P@1': 0.0,
              'P@5': 0.0,
              'P@10': 0.0,
              'R@20': 0.0,
              'R@50': 0.0,
              'R@100': 0.0,
              'nDCG@5': 0.0},
             'E05': {'P@1': 0.0,
              'P@5': 0.0,
           

### **References**


* [PyTerrier  retrieval and evaluation notebook](https://github.com/terrier-org/pyterrier/blob/master/examples/notebooks/retrieval_and_evaluation.ipynb).
*   [PyTerrier documentation.](https://pyterrier.readthedocs.io/_/downloads/en/latest/pdf/)

* Tamer Elsayed, Lectures notes and Labs, Qatar University  
